In [1]:
from typing import IO
import numpy as np

import glob, os, sys, json
import pdb

from cycler import cycler
import matplotlib as mpl
from numpy.core.fromnumeric import mean

import pandas

from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import ConnectionPatch

import matplotlib
from matplotlib import rc
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

from scipy.interpolate import interp1d
from scipy.stats import kstest
from scipy.stats import norm, poisson
from scipy.signal import convolve

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS
from astropy.time import Time

from IPython import embed

# Local
sys.path.append(os.path.abspath("../../Analysis/py"))
import alopeke_defs
import alopeke_utils2
import alopeke_analy

### Estimating the optical fluence ratio for the Galactic Magnetar

In [2]:
# Expected optical emission from SGR1935+2154, from FRB200428
# From this paper: https://arxiv.org/pdf/2007.02978.pdf
# Expect frequency dependence on fluence is nu^-0.46
spectral_index = -0.46

# STARE2 burst for 200428 was at 1.4 GHz and had a fluence of 1.5e6 Jy ms
radio_freq = 1.4e9
radio_fluence = 1.5e6



# Estimate what the optical fluence would be at 7000 angstroms
# Fluence = F_0 * nu^-0.46 -> F_0 = Fluence / (nu^-0.46)
optical_freq = 2.998e18 / (7000)
optical_fluence = (radio_fluence / radio_freq**spectral_index) * optical_freq ** spectral_index

# Distance to SGR1935+2154 based on association with SNR G57.2+0.8
# https://iopscience.iop.org/article/10.3847/2041-8213/aba262
sgr_distance = 9.0e3

sgr_isotropic_radio_energy = 4*np.pi*(sgr_distance * 3.08568025e18)**2 * radio_fluence * 1.0e-26 * radio_freq

sgr_fluence_ratio = optical_fluence/radio_fluence

print('Isotropic equivalent radio energy is:',sgr_isotropic_radio_energy)
print('Optical fluence is:',optical_fluence,'Jy ms')
print('Optical/radio fluence ratio is:',optical_fluence/radio_fluence)

Isotropic equivalent radio energy is: 2.03524182621986e+35
Optical fluence is: 4494.824141336007 Jy ms
Optical/radio fluence ratio is: 0.0029965494275573383


### Getting optical fluence ratios of Galactic optical pulsars

In [3]:
from astropy.table import Table

all_pulsar_data = []

# Crab Pulsar
data = Table.read('data/crab_data.csv',format='csv')
optical_idx = np.argmin(np.abs(data['frequency']-2.998e18/7000))
radio_idx = np.argmin(np.abs(data['frequency']-400e6))
optical_fluence=data[optical_idx]['fluence']/data[optical_idx]['frequency']
radio_fluence=data[radio_idx]['fluence']/data[radio_idx]['frequency']
fluence_ratio=optical_fluence/radio_fluence
radio_energy=4*np.pi*(2.0e3*3.08568025e18)**2 * data[radio_idx]['fluence']*300e-6

all_pulsar_data.append({'name':'crab','radio_freq':400.0e6,'optical_freq':2.998e18/7000,
                        'optical_fluence':optical_fluence,'radio_fluence':radio_fluence,
                        'fluence_ratio':optical_fluence/radio_fluence,
                        'distance':2.0e3,
                        'duration':300.0e-6,
                        'radio_energy':4*np.pi*(2.0e3*3.08568025e18)**2 * data[radio_idx]['fluence']*300e-6})

# Geminga
data = Table.read('data/geminga_data.csv',format='csv')
optical_idx = np.argmin(np.abs(data['frequency']-2.998e18/7000))
radio_idx = np.argmin(np.abs(data['frequency']-400e6))
optical_fluence=data[optical_idx]['flux']
radio_fluence=data[radio_idx]['flux']
fluence_ratio=optical_fluence/radio_fluence
radio_energy=4*np.pi*(250.0*3.08568025e18)**2 * data[radio_idx]['flux']*1.0e-6*1.0e-23*400.0e6*300e-3

all_pulsar_data.append({'name':'geminga','radio_freq':400.0e6,'optical_freq':2.998e18/7000,
                        'optical_fluence':optical_fluence,'radio_fluence':radio_fluence,
                        'fluence_ratio':optical_fluence/radio_fluence,
                        'distance':250.0,
                        'duration':300.0e-3,
                        'radio_energy':4*np.pi*(250.0*3.08568025e18)**2 * data[radio_idx]['flux']*1.0e-6*1.0e-23*400.0e6*300e-3})

# Vela Pulsar
data = Table.read('data/vela_data.csv',format='csv')
optical_idx = np.argmin(np.abs(data['frequency']-2.998e18/7000))
radio_idx = np.argmin(np.abs(data['frequency']-400e6))
optical_fluence=data[optical_idx]['flux']
radio_fluence=data[radio_idx]['flux']
fluence_ratio=optical_fluence/radio_fluence
radio_energy=4*np.pi*(294.0*3.08568025e18)**2 * data[radio_idx]['flux']*1.0e-6*1.0e-23*400.0e6*300e-3

all_pulsar_data.append({'name':'vela','radio_freq':400.0e6,'optical_freq':2.998e18/7000,
                        'optical_fluence':optical_fluence,'radio_fluence':radio_fluence,
                        'fluence_ratio':optical_fluence/radio_fluence,
                        'distance':294.0,
                        'duration':300.0e-3,
                        'radio_energy':4*np.pi*(294.0*3.08568025e18)**2 * data[radio_idx]['flux']*1.0e-6*1.0e-23*400.0e6*300e-3})

### Plotting empirical and theoretical data

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import rc
from matplotlib.ticker import MultipleLocator,AutoMinorLocator
from matplotlib.colors import ListedColormap

def ccolor(r,g,b):
    return((r/255.,g/255.,b/255., 1.0))

lightblue=ccolor(135, 206, 235)
red=ccolor(255,0,0)
blue=ccolor(0,0,255)
black=ccolor(0,0,0)
red=ccolor(255,0,0)
blue=ccolor(10,0,255)
green=ccolor(12,83,0)
magenta=ccolor(204,0,204)
goldenrod=ccolor(239,139,8)
orange=ccolor(204,102,0)
lightred=ccolor(255,178,178)

figsize=10
rc('font',**{'family':'serif','serif':['Times'],'size':5.0*figsize})
rc('text', usetex=True, color='k')

fig, ax = plt.subplots()
for i in ax.spines.keys(): ax.spines[i].set_linewidth(0.6*figsize)
fig.set_size_inches(1.8*figsize, 1.5*figsize)

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.tick_params(direction='in', length=2*figsize,
            width=0.6*figsize, which='major', axis='both', colors=black,
            pad=2*figsize, top=True, bottom=True, left=True, right=True)
ax.tick_params(direction='in', length=figsize,
            width=0.6*figsize, which='minor', axis='both', colors=black,
            pad=0.4*figsize, top=True, bottom=True, left=True, right=True)

# Burst radio energies at 400 MHz and our limits
radio_freq=400e6
burst1=2.6 # Jy ms
burst2=3.5 # Jy ms
frb_distance=150e6
burst1_energy=4*np.pi*(frb_distance * 3.08568025e18)**2 * burst1 * 1.0e-26 * radio_freq
burst2_energy=4*np.pi*(frb_distance * 3.08568025e18)**2 * burst2 * 1.0e-26 * radio_freq

# Optical fluence ratios
burst1_r_ratio=0.0041
burst1_i_ratio=0.0032
burst2_r_ratio=0.0071
burst2_i_ratio=0.0022

# Plot all limits as downward arrows
i=0
for energy,ratio,color,error in zip([burst1_energy,burst1_energy,burst2_energy,burst2_energy],
                        [burst1_r_ratio,burst1_i_ratio,burst2_r_ratio,burst2_i_ratio],
                        [blue,red,blue,red],
                        [0.8/burst1,0.8/burst1,0.8/burst2,0.8/burst2]):
    if i==0:
        label='r-band Limit'
    elif i==1:
        label='i-band Limit'
    else:
        label=None
    ax.quiver(energy,ratio,0,-0.01,color=color,edgecolor='k',linewidth=2,scale=0.01,angles='xy',scale_units='inches')#,
            #head_length=0.1*ratio,head_width=0.5*energy)
    ax.errorbar([energy],[ratio],xerr=[2*error*energy],yerr=0,uplims=[0],color=color,
               linewidth=0.4*figsize)
    i=i+1

label='Optical Pulsars'
for pulsar in all_pulsar_data:
    radio_energy = pulsar['radio_energy']
    fluence_ratio = pulsar['fluence_ratio']
    name = pulsar['name']
    uppername = name[0].upper() + name[1:]
    name = uppername
    ax.scatter([radio_energy],[fluence_ratio],marker='o',s=[300*figsize],
               color=orange,edgecolor='k',linewidth=1,label=label)
    ax.annotate(name, xy=(float(radio_energy)*10**(1.5), fluence_ratio), xycoords='data',
                fontsize=2.0*figsize)
    label = None

# SGR1935
ax.scatter([sgr_isotropic_radio_energy],[sgr_fluence_ratio],marker='*',s=[300*figsize],
           color=lightblue,edgecolor='k',linewidth=1,label='SGR 1935+2154')

# Model comparison - from https://arxiv.org/pdf/1905.02429.pdf
ax.text(3e35,4.5e-2,'Pulsar Magnetosphere',color=black,va='center',ha='center',fontsize=4.0*figsize)
ax.hlines(3e-2,1e34,1e44,linestyle='dashed',color=black,linewidth=0.5*figsize,zorder=0)

ax.text(7e33,3e-3,'Young SNR',color=orange,va='center',ha='right',fontsize=4.0*figsize)
ax.hlines(3e-3,1e34,1e44,linestyle='dashed',color=orange,linewidth=0.5*figsize,zorder=0)

ax.text(7e33,2e-6,'Maser Outflow Emission',color=magenta,va='center',ha='right',fontsize=4.0*figsize)
ax.hlines(2e-6,1e34,1e44,linestyle='dashed',color=magenta,linewidth=0.5*figsize)

ax.text(2.5e33,0.006,'Our Limits',color='k')

# Finalize plot and save

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim([1e21,1e39])
ax.set_ylim([1.0e-7,0.2])

ax.set_ylabel('Optical-to-Radio Fluence Ratio')
ax.set_xlabel('Burst Radio Energy (erg)')

plt.legend(fontsize=4.0*figsize,loc='upper left')
plt.tight_layout()
plt.savefig('fluence_ratio.png', format='png')